<img src="Scoreboard.jpg" width=800 height=1900>

<h1><center> Predicting College experiences Using College Scorecard <center></h1>

<center><b>Zion Joseph, Kelli Webber</b></center>

<h1><center>Introduction</center></h1>
<h4>Placeholder</h4>
<h1><center>Pulling The Data</center></h1>
<h2>Importing libaries</h2>

In [52]:
%%capture
!pip install folium
import folium
import json
import os
import requests
import pandas as pd
import numpy as np
from folium.plugins import HeatMap
import matplotlib.pyplot as plt
from sklearn import linear_model
import warnings
warnings.filterwarnings('ignore')

<h2>Pulling The Data from CSV</h2>

In [53]:
odata = pd.read_csv("Most-Recent-Cohorts-All-Data-Elements.csv")

odata.head()

,UNITID,OPEID,OPEID6,INSTNM,CITY,STABBR,ZIP,ACCREDAGENCY,INSTURL,NPCURL,...,OMAWDP6_FTNFT_POOLED_SUPP,OMAWDP8_FTNFT_POOLED_SUPP,OMENRYP8_FTNFT_POOLED_SUPP,OMENRAP8_FTNFT_POOLED_SUPP,OMENRUP8_FTNFT_POOLED_SUPP,OMAWDP6_PTNFT_POOLED_SUPP,OMAWDP8_PTNFT_POOLED_SUPP,OMENRYP8_PTNFT_POOLED_SUPP,OMENRAP8_PTNFT_POOLED_SUPP,OMENRUP8_PTNFT_POOLED_SUPP
0,100654,100200,1002,Alabama A & M University,Normal,AL,35762,Southern Association of Colleges and Schools C...,www.aamu.edu/,www2.aamu.edu/scripts/netpricecalc/npcalc.htm,...,0.4694,0.5136,0.0034,0.1429,0.3401,0.3333,0.3333,0,0,0.6667
1,100663,105200,1052,University of Alabama at Birmingham,Birmingham,AL,35294-0110,Southern Association of Colleges and Schools C...,www.uab.edu,uab.studentaidcalculator.com/survey.aspx,...,0.6117,0.624,0.0075,0.2166,0.1519,0.3642,0.4,0.0434,0.2755,0.2811
2,100690,2503400,25034,Amridge University,Montgomery,AL,36117-3553,Southern Association of Colleges and Schools C...,www.amridgeuniversity.edu,www2.amridgeuniversity.edu:9091/,...,0.5503,0.5562,0.0059,0.4379,0,0.4429,0.4714,0.0429,0.4857,0
3,100706,105500,1055,University of Alabama in Huntsville,Huntsville,AL,35899,Southern Association of Colleges and Schools C...,www.uah.edu,finaid.uah.edu/,...,0.5456,0.5651,0.0123,0.2841,0.1385,0.3378,0.3547,0.0236,0.4257,0.1959
4,100724,100500,1005,Alabama State University,Montgomery,AL,36104-0271,Southern Association of Colleges and Schools C...,www.alasu.edu,www.alasu.edu/cost-aid/forms/calculator/index....,...,0.4105,0.429,0.0062,0.216,0.3488,0.2931,0.3103,0.0345,0.3966,0.2586


<h2>Tidying of Columns</h2>

In [54]:
# Choose specific columns to display
data = odata[['INSTNM', 'CITY', 'STABBR', 'LONGITUDE', 'LATITUDE', 'MENONLY', 'WOMENONLY','ADM_RATE', 'TUITIONFEE_IN', 'TUITIONFEE_OUT', 'UGDS']]
# made a sample of the orignal data set
ndata = data.sample(n = 1000)
ndata.head()

,INSTNM,CITY,STABBR,LONGITUDE,LATITUDE,MENONLY,WOMENONLY,ADM_RATE,TUITIONFEE_IN,TUITIONFEE_OUT,UGDS
6985,Villanova University - Summer Music Studies Pr...,Villanova,PA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
793,Orange Technical College-Westside Campus,Winter Garden,FL,-81.571532,28.559574,0.0,0.0,NaN,NaN,NaN,412.0
6868,PJ's College of Cosmetology- Louisville,Louisville,KY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6763,Humphreys College-Modesto,Modesto,CA,NaN,NaN,NaN,NaN,NaN,14004.0,NaN,NaN
4541,College of Menominee Nation,Keshena,WI,-88.625628,44.859044,0.0,0.0,NaN,6200.0,6200.0,303.0


In [55]:
sex = []
for index, row in ndata.iterrows():
    if row['MENONLY'] > row['WOMENONLY']:
        sex.extend(["Men"])
    elif row['MENONLY'] == row['WOMENONLY']:
        sex.extend(["Co-ed"])
    else:
         sex.extend(["Women"])
ndata['SEX'] = sex
ndata = ndata[['INSTNM', 'CITY', 'STABBR', 'LONGITUDE', 'LATITUDE', 'SEX', 'TUITIONFEE_IN', 'TUITIONFEE_OUT', 'UGDS']]
ndata.head()

,INSTNM,CITY,STABBR,LONGITUDE,LATITUDE,SEX,TUITIONFEE_IN,TUITIONFEE_OUT,UGDS
6985,Villanova University - Summer Music Studies Pr...,Villanova,PA,NaN,NaN,Women,NaN,NaN,NaN
793,Orange Technical College-Westside Campus,Winter Garden,FL,-81.571532,28.559574,Co-ed,NaN,NaN,412.0
6868,PJ's College of Cosmetology- Louisville,Louisville,KY,NaN,NaN,Women,NaN,NaN,NaN
6763,Humphreys College-Modesto,Modesto,CA,NaN,NaN,Women,14004.0,NaN,NaN
4541,College of Menominee Nation,Keshena,WI,-88.625628,44.859044,Co-ed,6200.0,6200.0,303.0


<h2>Visualization</h2>

In [56]:
map_osm = folium.Map(location=[39.29, -100.61], zoom_start=4)

noNaN = ndata.dropna(subset = ['LATITUDE', 'LONGITUDE'])

for index, row in noNaN.iterrows():   
    color= 'black'
    folium.Circle(
    tooltip=row['INSTNM'],
    radius=40000,
    location=[row['LATITUDE'], row['LONGITUDE']],
    color=color,
    fill=True,
).add_to(map_osm)
    
map_osm